In [7]:
from langchain.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI

LLM_MODEL = "gpt-4o-mini"
TEXT_FILEPATH = "youth.txt"


llm = ChatOpenAI(temperature=0, model=LLM_MODEL)

In [8]:
with open(TEXT_FILEPATH, "r") as file:
    text = file.read()

# Forma mais simples - Prompt Único

##### Certifique-se que o texto pode ser resumido com apenas um prompt

In [24]:
import tiktoken

encoding = tiktoken.encoding_for_model(LLM_MODEL)
tokens = encoding.encode(text)

print(f"Esse texto tem {len(tokens)} tokens.")

Esse texto tem 17784 tokens.


In [21]:
from langchain.prompts import PromptTemplate

assert len(tokens) < 128_000

text_template = (
    "Conte a história seguinte na forma de um conto:\n"
    "```\n"
    "{text}"
    "\n```"
)

prompt = PromptTemplate(input_variables=["text"], template=text_template)

chain = prompt | llm

with get_openai_callback() as cb:
    summary = chain.invoke({"text": text})
    print(cb)

Tokens Used: 18813
	Prompt Tokens: 17804
	Completion Tokens: 1009
Successful Requests: 1
Total Cost (USD): $0.0032759999999999994


In [23]:
from IPython.display import Markdown
Markdown(summary.content)

**A Juventude e o Mistério das Criaturas Estranhas**

Era uma manhã tranquila no campo, e o sol mal começava a despontar por trás das colinas. Red e Slim, dois garotos cheios de energia e curiosidade, estavam prestes a viver uma aventura que mudaria suas vidas para sempre. Na noite anterior, um estrondo misterioso ecoara pelo céu, e agora, com a luz do dia, eles estavam determinados a descobrir o que havia acontecido.

"Você ouviu aquilo, Slim?" Red sussurrou, sua voz carregada de excitação. "Acho que encontrei algo incrível!"

Slim, ainda sonolento, pulou da cama e correu até a janela. "O que você encontrou?" perguntou, seus olhos brilhando de curiosidade.

"Venha comigo!" Red exclamou, puxando Slim para fora da casa. Eles correram pelo campo, a grama molhada da manhã fazendo cócegas em seus pés descalços. O mundo parecia ser deles, um vasto playground onde a imaginação não tinha limites.

Ao chegarem ao celeiro, Red parou abruptamente. "Estão aqui!" Ele apontou para uma grande jaula coberta com uma lona. "Olhe!"

Slim hesitou, um frio na barriga. "O que são essas coisas, Red?"

"São animais! Estranhos, mas incríveis. Eu os encontrei esta manhã. Vamos ver!" Red levantou a lona, revelando duas pequenas criaturas que se moviam rapidamente, com olhos brilhantes e corpos cobertos de pelos estranhos.

"Elas parecem... diferentes," Slim murmurou, um misto de fascínio e medo. "Você tem certeza de que elas são seguras?"

"Claro! Vamos alimentá-las. Um circo precisa de animais, e nós podemos ser os donos do maior circo do mundo!" Red disse, sua imaginação disparando.

Os meninos passaram a manhã cuidando das criaturas, alimentando-as com carne crua que Red havia pegado da cozinha. Mas, à medida que o dia avançava, a preocupação começou a se instalar. O que aconteceria se seus pais descobrissem? E se as criaturas não fossem apenas animais, mas algo mais?

Enquanto isso, os pais de Red e Slim, o Industrialista e o Astrônomo, estavam em uma conversa séria sobre o futuro da humanidade. O Astrônomo falava sobre a necessidade de mudança e inovação, enquanto o Industrialista se mostrava cético, preocupado com as consequências de um mundo que já havia enfrentado guerras devastadoras.

"Precisamos de juventude, de novas ideias," disse o Astrônomo. "Esses meninos representam a esperança de um futuro melhor."

Mas a esperança estava prestes a ser testada. Quando a mãe de Red entrou no celeiro, seu grito ecoou pelo ar. "O que são essas coisas?!" Ela exclamou, horrorizada ao ver as criaturas.

Os meninos, apavorados, tentaram explicar, mas a confusão tomou conta. O Industrialista e o Astrônomo correram para o celeiro, e a tensão aumentou. As criaturas, percebendo o pânico, começaram a emitir sons estranhos, como se tentassem se comunicar.

"Elas não são perigosas!" Red gritou, mas sua voz se perdeu na agitação. O Astrônomo, percebendo que as criaturas eram mais do que simples animais, tentou se concentrar e estabelecer uma conexão mental com elas.

"Estamos aqui para ajudar," ele pensou, e, para sua surpresa, as criaturas responderam. Elas eram seres de outro mundo, perdidos e confusos, mas não hostis.

A revelação trouxe um novo entendimento. O Industrialista, inicialmente cético, viu a oportunidade de um novo começo. "Se eles não nos querem mal, talvez possamos aprender com eles," disse ele, olhando para o Astrônomo.

Com a ajuda dos meninos, que se tornaram os embaixadores entre os mundos, os adultos começaram a entender a importância da juventude e da inovação. As criaturas, que antes eram vistas como estranhas e assustadoras, tornaram-se símbolos de esperança e mudança.

E assim, Red e Slim, com suas almas jovens e corajosas, não apenas salvaram as criaturas, mas também ajudaram a abrir os olhos de seus pais para um futuro onde a curiosidade e a imaginação poderiam levar a humanidade a novos horizontes. O circo que sonhavam em criar agora parecia uma possibilidade real, não apenas como um espetáculo, mas como um símbolo de união entre mundos.

A juventude, com sua capacidade de sonhar e explorar, havia se tornado a chave para um novo começo. E, enquanto o sol se punha no horizonte, Red e Slim sabiam que suas aventuras estavam apenas começando.

# Multiplos Prompts

In [32]:
from langchain_community.document_loaders.text import TextLoader

docs = TextLoader(file_path = TEXT_FILEPATH).load()

In [35]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# we need to split the long input text
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=100
)

split_docs = text_splitter.split_documents(docs)

In [37]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm, chain_type="refine")

with get_openai_callback() as cb:
    summary = chain.invoke(split_docs)
    print(cb)

Tokens Used: 55853
	Prompt Tokens: 37430
	Completion Tokens: 18423
Successful Requests: 22
Total Cost (USD): $0.0166683


In [42]:
from langchain.prompts import PromptTemplate

text_template = (
    "Por favor, traduza o seguinte texto para português:\n"
    "```\n"
    "{text}"
    "\n```"
)

prompt = PromptTemplate(input_variables=["text"], template=text_template)

chain = prompt | llm

with get_openai_callback() as cb:
    summary_translated = chain.invoke({"text": summary["output_text"]})
    print(cb)

Tokens Used: 3012
	Prompt Tokens: 1352
	Completion Tokens: 1660
Successful Requests: 1
Total Cost (USD): $0.0011987999999999999


In [44]:
from IPython.display import Markdown
Markdown(summary_translated.content)

O eBook do Project Gutenberg "Juventude" de Isaac Asimov conta a história de dois meninos, Red e Slim, que descobrem um senso de aventura e amizade nas primeiras horas da manhã. Após ouvirem sons misteriosos de trovão na noite anterior, eles se encontram do lado de fora em uma paisagem tranquila e orvalhada, onde compartilham sua empolgação sobre o mundo ao seu redor. A narrativa captura a inocência e a camaradagem da infância enquanto os meninos exploram seus arredores e desfrutam da liberdade de estar longe da supervisão dos adultos. Enquanto caminham por uma estrada de concreto, Red provoca Slim com a ideia de um segredo relacionado a um circo, acendendo a imaginação de Slim. Red revela que encontrou dois pequenos animais incomuns, que ele acredita que poderiam levá-los a se juntar a um circo, aprofundando ainda mais seu vínculo e senso de aventura. A história ilustra lindamente os sonhos e aspirações da infância, bem como a emoção de embarcar em uma jornada compartilhada.

Enquanto isso, a narrativa também apresenta personagens adultos, como um Astrônomo e um Industrialista, que discutem as implicações de um projetor de pensamentos, insinuando uma conexão mais profunda entre as aventuras inocentes dos meninos e as complexidades da vida adulta. A conversa deles reflete sobre as mudanças sociais e o contraste entre a imaginação juvenil e o peso das preocupações adultas. O Astrônomo alerta que a sociedade está estagnando e caminhando para a destruição, enquanto o Industrialista expressa ceticismo sobre a mudança. Eles ponderam por que seres avançados ainda buscariam recursos básicos como carvão e petróleo, sugerindo um desconexão entre o avanço tecnológico e as necessidades da sociedade. Essa justaposição enriquece a história, destacando a tensão entre as esperanças da juventude e os fardos de um mundo que perdeu seu caminho. Em última análise, a narrativa sugere que a vitalidade da juventude, personificada por Red e Slim, pode conter a chave para revitalizar uma sociedade cansada, enfatizando a importância de nutrir a curiosidade e a aventura diante do cinismo adulto.

O contexto adicional introduz temas de exploração e as consequências do avanço tecnológico, como visto através das experiências do Explorador e do Mercador em um ambiente alienígena. As discussões deles sobre os desafios da viagem interestelar e a natureza das civilizações alienígenas ressaltam ainda mais o contraste entre a inocência das aventuras de Red e Slim e as complexidades enfrentadas pelos adultos em um mundo em rápida mudança. A situação perigosa enfrentada pelo Explorador e pelo Mercador, incluindo um pouso forçado e encontros com vida alienígena, serve como um lembrete contundente dos perigos e da imprevisibilidade da exploração. Essa narrativa em camadas convida os leitores a refletir sobre o equilíbrio entre os sonhos juvenis e as duras realidades da existência, reforçando a ideia de que o espírito de aventura e imaginação é essencial tanto para crianças quanto para adultos.

À medida que a história se desenrola, as travessuras brincalhonas dos meninos são espelhadas pela tensa situação do Explorador e do Mercador, que se veem capturados por uma criatura gigante em um planeta alienígena. As tentativas desesperadas deles de se comunicar e sobreviver destacam o contraste gritante entre a inocência despreocupada de Red e Slim e os desafios que ameaçam a vida enfrentados pelos adultos. O pânico do Mercador e o raciocínio calmo do Explorador ilustram as complexidades da vida adulta, onde decisões têm consequências significativas. Essa estrutura narrativa dual enfatiza a importância da imaginação e da aventura na infância, ao mesmo tempo em que reconhece o peso da responsabilidade e a imprevisibilidade do mundo adulto. Em última análise, "Juventude" de Asimov serve como um lembrete tocante da necessidade de preservar o espírito de exploração e maravilha, independentemente da idade.

As interações entre Red e seu pai, juntamente com as discussões entre o Astrônomo e o Industrialista, ilustram ainda mais a tensão entre a curiosidade juvenil e o ceticismo adulto. A descoberta pelos meninos do que acreditam ser uma nave espacial, juntamente com suas discussões sobre a natureza de outros mundos, destaca seu espírito imaginativo e contrasta fortemente com as preocupações mais pragmáticas dos adultos. A insistência brincalhona de Red em cuidar dos animais que encontraram, apesar das dúvidas de Slim, demonstra a inocência e a determinação da juventude. Enquanto isso, os adultos lutam com as implicações de suas próprias descobertas e as realidades de suas vidas, enfatizando a desconexão entre as aspirações da juventude e os fardos da vida adulta. As percepções do Astrônomo sobre a natureza das civilizações alienígenas e seu desinteresse pela Terra devido à abundância de mundos adequados em outros lugares enfatizam ainda mais esse tema, sugerindo que o futuro pode estar nas mãos do espírito imaginativo e aventureiro dos jovens.

Em um momento crucial, a travessura inocente de Red leva a um confronto com seu pai sobre os animais que ele pegou, ilustrando o choque entre seus impulsos juvenis e as responsabilidades impostas pela vida adulta. A preocupação do Industrialista com o bem-estar do cozinheiro e a compreensão do Astrônomo sobre a curiosidade de Red destacam as complexidades da vida adulta, onde decisões e ações têm consequências significativas. Esse momento serve para reforçar a exploração da narrativa sobre a tensão entre a natureza despreocupada da infância e o peso das expectativas adultas, sublinhando a importância de nutrir o espírito de aventura e imaginação diante dos desafios da vida. A urgência da necessidade de Slim de falar com Red sobre os animais e a revelação de sua potencial natureza alienígena aumentam ainda mais as apostas de sua aventura, enfatizando a inocência dos meninos e as consequências iminentes de suas ações. À medida que eles lidam com as implicações de suas descobertas, a história ilustra de forma tocante o delicado equilíbrio entre a curiosidade juvenil e as responsabilidades que vêm com o crescimento. O pânico dos meninos ao perceberem a seriedade de sua situação, enquanto confrontam os animais que inicialmente viam como meras curiosidades, sublinha a transformação de sua aventura inocente em um momento de medo e responsabilidade, encapsulando a essência da exploração de Asimov sobre a juventude e a vida adulta.

O novo contexto adiciona profundidade à narrativa, particularmente nas interações entre os adultos e as estranhas criaturas descobertas por Red e Slim. A descrição frenética da mãe sobre os animais e a preocupação imediata do Industrialista com a segurança deles destacam o medo e o mal-entendido que muitas vezes acompanham o desconhecido. As tentativas do Astrônomo de se comunicar com as criaturas revelam as complexidades da vida adulta, onde conhecimento e compreensão são frequentemente ofuscados pelo medo e pela urgência. A realização do Industrialista de que as criaturas não são hostis, mas sim confusas e assustadas, enfatiza o tema da falta de comunicação entre gerações e a importância da empatia. Esse momento serve para ilustrar ainda mais o contraste entre a curiosidade inocente dos meninos e a tendência do mundo adulto de reagir com medo e agressão. Em última análise, "Juventude" de Asimov se torna uma rica tapeçaria de temas, entrelaçando a inocência da infância, as complexidades da vida adulta e a vital importância da imaginação e compreensão para unir os dois. A história culmina em um momento tocante, enquanto Red, personificando o espírito da juventude, observa a nave que parte, cheio de arrependimento por uma oportunidade perdida, simbolizando a natureza agridoce de crescer e o poder duradouro da imaginação.